In [1]:
#-----------------------------------------------------------------------------------------------------
# 문서들을 embed하는 예제
# - 여기서는 문서들을 전처리하고, 임베딩 하는 과정임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
#-----------------------------------------------------------------------------------------------------

import os
import random
import numpy as np
import pandas as pd
import time
import random

from utils import MyUtils
from utils import get_sentences

myutils = MyUtils(yam_file_path='./data/settings.yaml')
myutils.seed_everything()  # seed 설정
DEVICE = myutils.GPU_info() # GPU 혹은 CPU

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

False
device: cpu


In [2]:
# DATA_FOLDER에 파일들을 불러와서 DF로 만듬.
# -문서는 \n으로 구분해야 하며, 맨앞에는 title이 와야 하고, \n 다음 문단들이 와야함. 문단들은 \n 구분됨.
# -예: 회사 개요\n{회사내용}\n{제품구성}

# 파일이 여러개인 경우 폴더 지정
#DATA_FOLDER = './doc/사규개정-out-renew/'
DATA_FOLDER = './tmp/company/'

files = myutils.getListOfFiles(DATA_FOLDER)
assert len(files) > 0 # files가 0이면 assert 발생
print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

# 파일 경로를 지정하면됨.
#files = ["../data11/mpower_doc/신입사원교육.txt",]
docid = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.
count = 0

titles = []
contextids = []
contexts = []

for idx, file_path in enumerate(files):
    if '.ipynb_checkpoints' not in file_path:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            #print(data)
            data_list = data.split('\n\n')  # '\n\n' 구분으로 다락 구분
            docid += 1
            count = 0
            # titles, contextids 임의로 구해서 데이터 프레임 만듬.
            for i in range(len(data_list)):
                if i > 0 and data_list[i]:
                    count += 1
                    titles.append(data_list[0].strip())  # 문서 제목은 맨처음 문장이고 \n\n로 구분되어야 함.
                    contextids.append(f'{docid}_{count}')
                    contexts.append(data_list[0].strip() + '\n' + data_list[i].strip()) # 두번째는 문서 제목+문서내용 합처서 문장만듬.
 
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, titles, contextids)), columns = ['context','question', 'contextid'])     

print(f'*문단 총 계수:{len(df_contexts)}\n')

*file_count: 46, file_list:['./tmp/company/㈜모코엠시스 보안관리규정.txt', './tmp/company/임원퇴직금 지급규정.txt', './tmp/company/외근업무 지원비 규정.txt', './tmp/company/개인정보 내부 관리계획.txt', './tmp/company/기념일 축하제도 지원 지침.txt']
*문단 총 계수:781



In [3]:
# JSON 파일로 저장해둠.
import json
examples = []

for context, title, contextid in zip(contexts, titles, contextids):
    doc = {}
    doc['context'] = context  # 문단
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("./data/moco_context.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

In [4]:
# 문장들로 분리
doc_sentences = get_sentences(df=df_contexts, remove_sentnece_len=1, remove_duplication=False)

print(f'len:{len(doc_sentences)}, 1: {doc_sentences[1]}')

  0%|          | 0/781 [00:00<?, ?it/s][Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class

100%|██████████| 781/781 [01:55<00:00,  6.76it/s]

*[get_sentences] 문장처리=>len:3, time:115.4611
*[get_sentences] 문장 길이=>평균:6.647887323943662 / MAX: 19 / MIN: 1

len:781, 1: ['모코엠시스 보안관리규정', '제2조 (정 의) 이 규정에서 사용하는 용어의 정의는 다음 각 호와 같다.', '전산 자료라 함은 전산 기기로 출력되거나 저장된 유,무형의 자료를 말한다.', '이에는 전산 기기상의 데이터 뿐만 아니라 웹 또는 네트워크 상에 전송되는 자료를 포함한다.', '전산 기기라 함은 전산 자료를 읽을 수 있는 하드웨어와 이에 준하는 것을 말한다.', '이에는 개인용 PC, 랩탑 PC(Notebook), 서버용 PC 뿐만 아니라 플로피 디스크, USB 메모리 및 저장장치(이동 HDD), CDR 및 이에 준하는 광 미디어 디스크 등을 포함한다.', '통신 장비라 함은 전산 기기들이 전산 자료를 전송하거나 공유를 할 수 있는 하드웨어 장비 및 이에 준하는 것을 말한다.', '이에는 라우터, Firewall, 스위칭 허브, 보안 서버 등을 포함한다.', '전산 시스템이라 함은 당사의 회계인사시스템, 메일 시스템, KMS, 그룹웨어 및 보안 시스템 등을 말한다.']


In [5]:
# 인덱스 추가
from tqdm.notebook import tqdm
from utils import embed_text, bi_encoder, mpower_index_batch, create_index, clustering_embedding, kmedoids_clustering_embedding

# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
    
# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#---------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#---------------------------------------------------------------------------
def index_data(es, df_contexts, doc_sentences:list):
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_contexts['contextid'].values.tolist()
    rfile_texts = df_contexts['context'].values.tolist()

    if OUT_DIMENSION == 0:
        dimension = 768
    else:
        dimension = 128

    clustering_num = NUM_CLUSTERS
        
    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences-------------------')
            if len(sentences) > 5:
                print(sentences[:5])
            else:
                print(sentences)
                
            myutils.log_message(f'*[index_data] embeddings.shape: {embeddings.shape}', log_folder='./log/documet')
            print()
        
        #----------------------------------------------------------------
        multiple = 1
        
        # [bong][2023-04-28] 임베딩 출력 계수에 따라 클러스터링 계수를 달리함.
        if NUM_CLUSTERS_VARIABLE == True:
            embeddings_len = embeddings.shape[0]
            if embeddings_len > 2000:
                multiple = 6
            elif embeddings_len > 1000:
                multiple = 5 # 5배
            elif embeddings_len > 600:
                multiple = 4 # 4배
            elif embeddings_len > 300:
                multiple = 3 # 3배
            elif embeddings_len > 100:
                multiple = 2 # 2배
        #----------------------------------------------------------------
        
        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                # [bong][2023-04-28] 문장이 많은 경우에는 클러스터링 계수를 2,3배수로 함
                emb = clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
            clustering_num = 1  # 평균값일때는 NUM_CLUSTERS=1로 해줌.
        # 2=문장임베딩
        else:
            emb = embeddings

        if i < 3:
            myutils.log_message(f'*[index_data] cluster emb.shape: {emb.shape}', log_folder='./log/documet')
            print()
        
        #--------------------------------------------------- 
        # docs에 저장 
        #  [bong][2023-04-28] 여러개 벡터인 경우에는 벡터를 10개씩 분리해서 여러개 docs를 만듬.
        for j in range(multiple):
            count += 1
            doc = {}                                #dict 선언
            doc['rfile_name'] = rfile_names[i]      # contextid 담음
            doc['rfile_text'] = rfile_texts[i]      # text 담음.
            doc['dense_vectors'] = emb[j * clustering_num : (j+1) * clustering_num] # emb 담음.
            docs.append(doc)
        #---------------------------------------------------    

            if count % BATCH_SIZE == 0:
                mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
                docs = []
                myutils.log_message("[index_data](1) Indexed {} documents.".format(count), log_folder='./log/documet')
                 

    if docs:
        mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
        myutils.log_message("[index_data](2) Indexed {} documents.".format(count), log_folder='./log/documet')

    es.indices.refresh(index=ES_INDEX_NAME)

    myutils.log_message(f'*인덱싱 시간 : {time.time()-start:.4f}\n', log_folder='./log/documet')
    print()
#---------------------------------------------------------------------------


In [6]:
# param--------------------------------------------------------------------
OUT_DIMENSION =128   # 128 혹은 768이면 0입력
EMBEDDING_METHOD=1  # 0=클러스터링 임베딩, 1=문장평균임베딩, 2=문장임베딩
NUM_CLUSTERS=10     # 클러스터링 임베딩일때 클러스터링 수 
NUM_CLUSTERS_VARIABLE=False # 클러스터링 임베딩일때 클러스터링을 문장계수마다 다르계할지.
CLUSTRING_MODE = "kmeans"  # "kmeans" = k-평균 군집 분석, kmedoids =  k-대표값 군집 분석
OUTMODE = "mean"           # kmeans 일때=>mean=평균임베딩값, max=최대임베딩값.,  kmedoids 일때 =>mean=평균임베딩값, medoid=대표임베딩값.

MODEL_PATH = './model/kpf-sbert-128d-v1'#'../../../data11/model/kpf-sbert-v1.1'
POLLING_MODE = 'mean' # 폴링모드 

FLOAT_TYPE = 'float16' # float32 혹은 float16
SEED = 111

# ES 접속
ES_URL = 'http://192.168.0.51:9200/'               # es 접속 주소
ES_INDEX_NAME = 'qaindex_128_avg_202403'          # 생성혹은 추가할 인덱스 명
ES_INDEX_FILE = './data/mpower10u_128d_1.json'   # 인덱스 구조 파일경로
BATCH_SIZE=20       # ES 배치 사이즈
CREATE_INDEX = True # True이면 기존에 인덱스가 있다면 제거하고 다시 생성.
# param--------------------------------------------------------------------

es = Elasticsearch(ES_URL)
create_index(es, ES_INDEX_FILE, ES_INDEX_NAME, create=CREATE_INDEX)

# 임베딩 모델 로딩
WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)
print()
try:
    index_data(es, df_contexts, doc_sentences)
except Exception as e:
    error = f'index_data fail'
    msg = f'{error}=>{e}'
    myutils.log_message(f'/embed/es {msg}', log_folder='./log/documet')

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 128, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)



  0%|          | 0/781 [00:00<?, ?it/s]

[0] sentences-------------------
['모코엠시스 보안관리규정', '제 1 장 총칙', '제1조 (목적) 이 규정은 모코엠시스 의 전산 자료, 전산 기기, 통신 장비, 전산 시스템 등이 타인에게 유출 또는 점거 시 자료 파괴와 혼란 및 불이익을 줄 수 있는 위해 상황으로부터 보호하기 위해 대외비 자료, 개인용 컴퓨터 보안, 통제 구역 등을 규정하여 보호함을 그 목적으로 한다.']


[1] sentences-------------------
['모코엠시스 보안관리규정', '제2조 (정 의) 이 규정에서 사용하는 용어의 정의는 다음 각 호와 같다.', '전산 자료라 함은 전산 기기로 출력되거나 저장된 유,무형의 자료를 말한다.', '이에는 전산 기기상의 데이터 뿐만 아니라 웹 또는 네트워크 상에 전송되는 자료를 포함한다.', '전산 기기라 함은 전산 자료를 읽을 수 있는 하드웨어와 이에 준하는 것을 말한다.']


[2] sentences-------------------
['모코엠시스 보안관리규정', '제 2 장 정보 보안 정책', '제3조 (개념)', '당사의 모든 전산 기기, 통신 장비 및 전산 시스템은 사내 보안 관리 규정(이하 보안규정이라 칭함.)에 의거 분류 및 관리 되어야 한다.', '유무형의 전산 자료를 포함하는 모든 정보들은 보안규정에 의한 세부 정책에 따라 분류 및 관리 되어야 한다.']




  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]